# Model transforming words into a number
- input: sequence of letters
- output: number 


Examples: 
- input: sequence 'one hundred' output: 100
- input: sequence 'two hundred fourteen' output: 214

The code in file number2words.py taken from: https://www.codesansar.com/python-programming-examples/number-words-conversion-no-library-used.htm

In [1]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
import numpy as np
from number2words import getWords

## Model

In [2]:
model = Sequential()
model.add(LSTM(128,input_shape=(None,1),return_sequences=True)) # sequences of singlen numbers
model.add(LSTM(128))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer="adam",metrics=['mae','mse'])
num_epochs = 0
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, None, 128)         66560     
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 198,273
Trainable params: 198,273
Non-trainable params: 0
_________________________________________________________________


## Dataset creation

### Helper methods

In [6]:
# helper method, converts sequence of numbers to text
def to_text(sample):
    return ''.join([idx2char[int(x)] for x in sample])
# helper method, converts text to sequence of numbers
def to_number(words):
    return np.array([char2idx[char] for char in words])

### Dataset - **samples** and **labels**

In [7]:
DATASET_SIZE=200

samples = []
labels = []
all_words = ''
max_len = 0
for i in range(DATASET_SIZE):
    labels.append(i)
    #words = lslownie(i)
    words = getWords(i)
    samples.append(words)
    all_words += words
    if len(words)>max_len: 
        max_len = len(words)
  
print('Max len of text',max_len)
vocab = sorted(set(all_words))
vocab_size = len(vocab)
print('vocabulary (used letters)',vocab)
print ('unique characters',vocab_size)

Max len of text 27
vocabulary (used letters) [' ', 'd', 'e', 'f', 'g', 'h', 'i', 'l', 'n', 'o', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
unique characters 19


#### Creating a mapping from unique characters to indices

In [8]:
char2idx = {char:index for index, char in enumerate(vocab)}
print('char2idx:\n',char2idx)
idx2char = np.array(vocab)
print('idx2char\n',idx2char)

char2idx:
 {' ': 0, 'd': 1, 'e': 2, 'f': 3, 'g': 4, 'h': 5, 'i': 6, 'l': 7, 'n': 8, 'o': 9, 'r': 10, 's': 11, 't': 12, 'u': 13, 'v': 14, 'w': 15, 'x': 16, 'y': 17, 'z': 18}
idx2char
 [' ' 'd' 'e' 'f' 'g' 'h' 'i' 'l' 'n' 'o' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y'
 'z']


#### Convert letters to numbers using char2idx

In [9]:
samples_int = []
for s in samples:
    v = np.array([char2idx[char] for char in s])
    samples_int.append(v) # different sizes!
print(samples[123],' ->becomes-> ',samples_int[123])

one hundred twenty three    ->becomes->  [ 9  8  2  0  5 13  8  1 10  2  1  0 12 15  2  8 12 17  0 12  5 10  2  2
  0  0]


#### From list of lists to numpy - must have a fixed number of characters (30 -> max_len)

In [10]:
samples = np.zeros((DATASET_SIZE,max_len))
for i in range(len(samples_int)):
    for j in range(len(samples_int[i])):
        samples[i,j] = np.array(samples_int[i][j]) # all not used have '0' which is ' '
print('SAMPLES\n\n',samples)
print(samples.shape)

SAMPLES

 [[18.  2. 10. ...  0.  0.  0.]
 [ 9.  8.  2. ...  0.  0.  0.]
 [12. 15.  9. ...  0.  0.  0.]
 ...
 [ 9.  8.  2. ...  0.  0.  0.]
 [ 9.  8.  2. ...  0.  0.  0.]
 [ 9.  8.  2. ...  0.  0.  0.]]
(200, 27)


In [11]:
samples = np.expand_dims(samples,axis=2) #add the third dimension
labels = np.array(labels,dtype=float)

print("Sample (for 123):\n",samples[123])
print("Sample decoded",to_text(samples[123]))
print("Label (output):",labels[123])

print('samples shape',samples.shape)
print('labels shape',labels.shape)

Sample (for 123):
 [[ 9.]
 [ 8.]
 [ 2.]
 [ 0.]
 [ 5.]
 [13.]
 [ 8.]
 [ 1.]
 [10.]
 [ 2.]
 [ 1.]
 [ 0.]
 [12.]
 [15.]
 [ 2.]
 [ 8.]
 [12.]
 [17.]
 [ 0.]
 [12.]
 [ 5.]
 [10.]
 [ 2.]
 [ 2.]
 [ 0.]
 [ 0.]
 [ 0.]]
Sample decoded one hundred twenty three   
Label (output): 123.0
samples shape (200, 27, 1)
labels shape (200,)


In [12]:
TRAINING_SIZE = .5
from sklearn.model_selection import train_test_split
(trainSamples, testSamples, trainLabels, testLabels) = train_test_split(samples, labels,train_size=TRAINING_SIZE, random_state=1)
print('Training samples:',len(trainSamples),' test samples',len(testSamples))

Training samples: 100  test samples 100


In [17]:
EPOCHS=100
BATCH_SIZE = int(len(trainSamples)/4)
print('Training with',len(trainSamples),'samples',EPOCHS,'epochs and batch_size=',BATCH_SIZE)
for x in range(10):
    H = model.fit(trainSamples, trainLabels, epochs=EPOCHS,verbose=0,batch_size=BATCH_SIZE)
    num_epochs += EPOCHS
    print("\n{}/10 Epochs: {} - loss={:6.3f}, loss improvement={:6.3f}".
          format(x, num_epochs,H.history['loss'][-1], H.history['loss'][0]-H.history['loss'][-1]))
    check_model()
print("Done")

Training with 100 samples 100 epochs and batch_size= 25

0/10 Epochs: 1100 - loss=20.782, loss improvement=37.244
text => [predicted value] error=[error]
seventy one                 =>  71.70 error = 0.70
one hundred eighty nine     =>  171.35 error = 17.65
one hundred sixty seven     =>  166.90 error = 0.10
one hundred forty           =>  138.55 error = 1.45
one hundred fifty eight     =>  166.43 error = 8.43
one hundred thirty eight    =>  140.75 error = 2.75
twenty three                =>  23.43 error = 0.43
one hundred sixty six       =>  162.71 error = 3.29
one hundred six             =>  111.67 error = 5.67
one hundred seventy         =>  171.00 error = 1.00
Mean error = 5.8145185

1/10 Epochs: 1200 - loss=10.194, loss improvement=10.290
text => [predicted value] error=[error]
one hundred sixty two       =>  156.66 error = 5.34
one hundred twenty two      =>  129.68 error = 7.68
forty four                  =>  54.97 error = 10.97
one hundred twenty eight    =>  128.84 error = 0.8

In [18]:
import random

def check_model(verbose=0,how_many=10):
    pred = model.predict(samples)
    print('text => [predicted value] error=[error]')
    error = []
    for i in range(len(pred)):
        res = samples[i]
        error.append(abs(i-pred[i]))
        if verbose==1:
            train = ''
            if i in trainLabels: train='[T]'
            print(i,to_text(res),'=> {:.2f} error = {:.2f}'.format(pred[i,0],abs(i-pred[i,0])),train)
    if verbose<1: # if not verbose just display 'how_many' random samples
        for i in range(how_many):        
            x = random.randrange(DATASET_SIZE)
            res = samples[x]
            print(to_text(res),'=>  {:.2f} error = {:.2f}'.format(pred[x,0],abs(x-pred[x,0])))      
    print('Mean error =',np.mean(error))        
    return np.mean(error)
check_model(1)

text => [predicted value] error=[error]
0 zero                        => 0.10 error = 0.10 [T]
1 one                         => 1.19 error = 0.19 [T]
2 two                         => 1.96 error = 0.04 [T]
3 three                       => 2.98 error = 0.02 [T]
4 four                        => 8.33 error = 4.33 
5 five                        => 7.55 error = 2.55 
6 six                         => 6.06 error = 0.06 [T]
7 seven                       => 6.92 error = 0.08 [T]
8 eight                       => 7.91 error = 0.09 [T]
9 nine                        => 9.07 error = 0.07 [T]
10 ten                         => 10.36 error = 0.36 [T]
11 eleven                      => 14.01 error = 3.01 
12 twelve                      => 10.02 error = 1.98 
13 thirteen                    => 26.89 error = 13.89 
14 fourteen                    => 46.07 error = 32.07 
15 fifteen                     => 14.69 error = 0.31 [T]
16 sixteen                     => 57.29 error = 41.29 
17 seventeen                 

4.392348

In [27]:

#x = to_number('one hundred thirty nine      ')
#x = to_number('one hundred ninety nine     ')
x = to_number('one                               ')

#              xxxxxxxxxxxxxxxxxxxxxxxxxxxxx
x = np.expand_dims(x,axis=1)
x = np.expand_dims(x,axis=0)
model.predict(x)

array([[-15.577038]], dtype=float32)

In [ ]:
model.save('model_words2numbers.h5')